### TWITTER RANKINGS WITH PAGERANK

In [13]:
import pandas as pd
import numpy as np

In [5]:
# a follows b 
twitter = pd.read_csv("./twitter_combined.txt", header= None)

In [6]:
twitter["id"] = twitter.iloc[:, 0].apply(lambda x: x.split(' ')[0])
twitter["follows"] = twitter.iloc[:, 0].apply(lambda x: x.split(' ')[1])

In [7]:
twitter = twitter[['id', 'follows']]

In [25]:
twitter_subsample = twitter[-round(len(twitter)/200):] # Take a subsample

In [26]:
edges = dict(twitter_subsample.groupby(['id']).apply(lambda x: list(x['follows'])))

In [27]:
nodes = list(set(list(twitter_subsample.id.unique()) + list(twitter_subsample.follows.unique())))

**1) CREATE ADJACENCY BINARY MATRIX (A)**

In [28]:
# Create n x n matrix, 1 if row i follows column j
A = []
t = 0
for i in nodes:
    following = []
    for j in nodes:
        # if person i follows anyone
        if i in edges:
            if j in edges[i]:
                following.append(1)
            else:
                following.append(0)
        # if person i doesn't follow anyone (DANGLING NODES) - let them be connected to everyone
        else:
            if i == j:
                following.append(0) # don't connect to itself
            else:
                following.append(1) # connect to all other nodes
    A.append(following)
    t += 1

** 2) MAKE (A) COLUMN STOCHASTIC**
* Divide each row by sum(row_i)
* Take the transpose

In [29]:
A = np.array(A)

In [30]:
A2 = []
for i in range(len(A)):
    sum_ = sum(A[i])
    if sum_ > 0:
        A2.append(A[i]  / sum_)
    else:
        A2.append(A[i])

In [31]:
A2 = np.array(A2).T

In [32]:
sum(A2, axis = 0)

array([[  0.00000000e+00,   9.64878425e-05,   9.64878425e-05, ...,
          9.64878425e-05,   9.64878425e-05,   9.64878425e-05],
       [  9.64878425e-05,   0.00000000e+00,   9.64878425e-05, ...,
          9.64878425e-05,   9.64878425e-05,   9.64878425e-05],
       [  9.64878425e-05,   9.64878425e-05,   0.00000000e+00, ...,
          9.64878425e-05,   9.64878425e-05,   9.64878425e-05],
       ..., 
       [  9.64878425e-05,   9.64878425e-05,   9.64878425e-05, ...,
          0.00000000e+00,   9.64878425e-05,   9.64878425e-05],
       [  9.64878425e-05,   9.64878425e-05,   9.64878425e-05, ...,
          9.64878425e-05,   0.00000000e+00,   9.64878425e-05],
       [  9.64878425e-05,   9.64878425e-05,   9.64878425e-05, ...,
          9.64878425e-05,   9.64878425e-05,   0.00000000e+00]])

**3) CREATE M = (1-p)A + pS (GOOGLE MATRIX)**

In [17]:
S = (np.ones(len(A2)**2)*(1/len(A2))).reshape(len(A2), len(A2))

In [18]:
p = .15
M = (1 - p)*A2 + p*S

In [19]:
v = (np.ones(len(A2))*(1/len(A2))).reshape(len(A2), 1)

In [20]:
x = v
x_prev = v
i = 0
diff_prev = 1e10
while True:
    x_prev = x
    x = M.dot(x) / sum(M.dot(x)) # L1 norm
    diff = sum(abs(x - x_prev))
    if diff == 0: # Check convergence for same eigenvector
        break
    if diff_prev == diff: # If error is converged 
        print(diff)
        break
    diff_prev = diff
    
    print(diff)   # Check convergence it is near 0 but never 0  

[ 0.00338467]
[  4.18671836e-05]
[  4.82953016e-07]
[  1.07064731e-08]
[  1.93764205e-10]
[  5.65411307e-12]
[  1.53091610e-13]
[  7.86137377e-15]
[  1.57536880e-14]
[  2.23078121e-14]
[  1.81789940e-14]
[  9.21840187e-15]
[  8.06102960e-15]
[  9.08131805e-15]
[  8.94855750e-15]
[  9.13125912e-15]
[  1.19812201e-14]
[  1.16502809e-14]
[  8.88388484e-15]
[  9.08673907e-15]
[  8.93293143e-15]
[  9.23902881e-15]
[  1.57200777e-14]
[  2.33826765e-14]
[  1.82543461e-14]
[  9.21100219e-15]
[  8.06128710e-15]
[  9.08131805e-15]
[  8.94855750e-15]
[  9.13125912e-15]
[  1.19812201e-14]
[  1.16502809e-14]
[  8.88388484e-15]
[  9.08673907e-15]
[  8.93293143e-15]
[  9.23902881e-15]
[  1.57200777e-14]
[  2.33826765e-14]
[  1.82543461e-14]
[  9.21100219e-15]
[  8.06128710e-15]
[  9.08131805e-15]
[  8.94855750e-15]
[  9.13125912e-15]
[  1.19812201e-14]
[  1.16502809e-14]
[  8.88388484e-15]
[  9.08673907e-15]
[  8.93293143e-15]
[  9.23902881e-15]
[  1.57200777e-14]
[  2.33826765e-14]
[  1.82543461e-14

[  9.08131805e-15]
[  8.94855750e-15]
[  9.13125912e-15]
[  1.19812201e-14]
[  1.16502809e-14]
[  8.88388484e-15]
[  9.08673907e-15]
[  8.93293143e-15]
[  9.23902881e-15]
[  1.57200777e-14]
[  2.33826765e-14]
[  1.82543461e-14]
[  9.21100219e-15]
[  8.06128710e-15]
[  9.08131805e-15]
[  8.94855750e-15]
[  9.13125912e-15]
[  1.19812201e-14]
[  1.16502809e-14]
[  8.88388484e-15]
[  9.08673907e-15]
[  8.93293143e-15]
[  9.23902881e-15]
[  1.57200777e-14]
[  2.33826765e-14]
[  1.82543461e-14]
[  9.21100219e-15]
[  8.06128710e-15]
[  9.08131805e-15]
[  8.94855750e-15]
[  9.13125912e-15]
[  1.19812201e-14]
[  1.16502809e-14]
[  8.88388484e-15]
[  9.08673907e-15]
[  8.93293143e-15]
[  9.23902881e-15]
[  1.57200777e-14]
[  2.33826765e-14]
[  1.82543461e-14]
[  9.21100219e-15]
[  8.06128710e-15]
[  9.08131805e-15]
[  8.94855750e-15]
[  9.13125912e-15]
[  1.19812201e-14]
[  1.16502809e-14]
[  8.88388484e-15]
[  9.08673907e-15]
[  8.93293143e-15]
[  9.23902881e-15]
[  1.57200777e-14]
[  2.3382676

KeyboardInterrupt: 

**4) IDENTIFY RANKS**

In [21]:
# Sorted in increasing order
x = [i[0] for i in x]
sorted_x = np.argsort(x)

In [22]:
# Index of worst 10, least popular people
worst_idx = sorted_x[:10]
# Index of best 10, most popular people
best_idx = sorted_x[-10:]

**5) EXRACT IDs OF TOP AND BOTTOM**

In [23]:
print(np.array(nodes)[worst_idx])

['86560711' '93787797' '88931752' '80479756' '89826562' '89032548'
 '94147192' '86664924' '83285096' '85716069']


In [24]:
print(np.array(nodes)[best_idx])

['7888452' '53235381' '8596922' '14944566' '403518639' '5538252' '7861312'
 '3829151' '14719129' '14645160']
